# XML Agent

Some language models (like Anthropic's Claude) are particularly good at reasoning/writing XML. This goes over how to use an agent that uses XML when prompting. 

In [1]:
from langchain import hub
from langchain.agents import AgentExecutor, create_xml_agent
from langchain_community.chat_models import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults

## Initialize Tools

We will initialize the tools we want to use

In [2]:
tools = [TavilySearchResults(max_results=1)]

## Create Agent

In [3]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/xml-agent-convo")

In [4]:
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

In [9]:
llm_with_stop = model.bind(stop=["</tool_input>"])

In [15]:
agent = (
    {
        "question": lambda x: x["question"],
        "agent_scratchpad": lambda x: format_xml(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_stop
    | XMLAgentOutputParser()
)

In [ ]:
from langchain.agents import AgentExecutor

In [16]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [18]:
agent_executor.invoke({"question": "what's the weather in New york?"})



> Entering new AgentExecutor chain...
 <tool>search</tool>
<tool_input>weather in new york32 degrees <tool>search</tool>
<tool_input>weather in new york32 degrees <final_answer>
The weather in New York is 32 degrees.
</final_answer>

> Finished chain.


{'question': 'what's the weather in New york?',
 'output': '\nThe weather in New York is 32 degrees.\n'}

## Run Agent

In [5]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [6]:
agent_executor.invoke({"input": "what's the weather in New york?"})



> Entering new AgentExecutor chain...
 <tool>tavily_search_results_json</tool><tool_input>what is LangChain?[{'url': 'https://aws.amazon.com/what-is/langchain/', 'content': 'What Is LangChain? What is LangChain?  How does LangChain work?  Why is LangChain important?  that LangChain provides to reduce development time.LangChain is an open source framework for building applications based on large language models (LLMs). LLMs are large deep-learning models pre-trained on large amounts of data that can generate responses to user queries—for example, answering questions or creating images from text-based prompts.'}] <final_answer>LangChain is an open source framework for building applications based on large language models (LLMs). It allows developers to leverage the power of LLMs to create applications that can generate responses to user queries, such as answering questions or creating images from text prompts. Key benefits of LangChain are reducing development time and effort compared t

{'input': 'what's the weather in New york?',
 'output': 'The weather in New York is 32 degrees'}